<a href="https://colab.research.google.com/github/bosbintang/numerical-method-2/blob/main/Numerical_method_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def damped_jacobi(v,b,a):
    w=2./3.
    v[1:-1]=(1.-w)*v[1:-1]+w*(1/(((h**2)*a[1:-1])+2.))*(b[1:-1]+v[:-2]+v[2:])
    
def gseidel(v,b,a):
    for i in range(1,len(v)-1):
        v[i]=(1./(2+a*h**2))*(b[i]+v[i-1]+v[i+1])   

def restrict(v):
    return np.pad(0.25*v[1:-3:2]+0.5*v[2:-2:2]+0.25*v[3:-1:2],1,'constant')

def prolong(v):
    r=np.zeros(2*len(v)-1)
    r[::2]=v
    r[1::2]=0.5*(v[:-1]+v[1:])
    return r

def amul(v,a):
    return np.pad(-v[:-2]+(2+a[1:-1]*h**2)*v[1:-1]-v[2:],1,'constant')

def vcycle(v,b,a):
    for i in range(3):
        damped_jacobi(v,b,a)
    if len(v) <= 3:
        return
    
    r=b-amul(v,a)
    r2=4.*restrict(r)
    e2=np.zeros_like(r2)
    a1=restrict(a)
    vcycle(e2,r2,a1)
    v+=prolong(e2)
    
    for i in range(3):
        damped_jacobi(v,b,a)

def tridiag_solver(b):
    b=np.copy(b)
    v=np.zeros_like(b)
    c=np.zeros_like(b)

    for i in range(1,len(v)-1):
        c[i]=-1./(2+c[i-1])
        b[i]=(b[i]+b[i-1])/(2+c[i-1])

    for i in reversed(range(1,len(v)-1)):
        v[i]=b[i]-c[i]*v[i+1]

    return v

for j in range(3):  
    N=2**(4+(2*(j+1)))
    h=1./N
    a=np.zeros(N+1)
    a[1:11]=(1000*N**2)
    x=np.linspace(0.,1.,N+1)
    b=np.full_like(x,2)
    b[0]=b[-1]=0
    ve=tridiag_solver(b)
    
    v=np.zeros_like(x)
    r=b-amul(v,a)

    for k in range(10):
        z=0.5*r
        ak=np.dot(r,z)/np.dot(amul(z,a),z)
        v=v+ak*z
        r1=r-ak*z
        z1=0.5*r1
        bk=np.dot(r1,z1)/np.dot(r,z)
        z=z1+bk*z
        r=z
    r_norm0=np.sqrt(r.dot(r))
    r_norm=[r_norm0]

    for i in range(100):
        vcycle(v,b,a)   
        r=b-amul(v,a)
        r_norm.append(np.sqrt(r.dot(r)))
    
    step=np.argmax(np.array(r_norm)<1e-03)
    
    print("N:",N)
    print("Steps: ",step)

N: 64
Steps:  14
N: 256
Steps:  22
N: 1024
Steps:  28
